# Imports

In [31]:
from snowflake.snowpark.session import Session 
import snowflake.snowpark.types as T 
import snowflake.snowpark.functions as F 
from snowflake.ml.modeling.preprocessing import OneHotEncoder



import warnings 
warnings.filterwarnings("ignore")


# Connect to Snowflake 

In [32]:
connection_params = {
    "connection_name": "default"
}

# To establish a connection to Snowflake, create a Snowpark session and pass connection_params as an argument
session = Session.builder.configs(connection_params).create()



In [33]:
# Find the current warehouse 

session.sql("SELECT CURRENT_WAREHOUSE()").collect()

[Row(CURRENT_WAREHOUSE()='JING_TEST_WH')]

# Prepare Data

In [40]:
session.sql("ALTER WAREHOUSE JING_TEST_WH SET WAREHOUSE_SIZE='LARGE'").collect()

[Row(status='Statement executed successfully.')]

In [41]:
session.use_database("ML_SNOWPARK_CI_CD_DB")
session.use_schema("DATA_PROCESSING")

In [42]:
session.sql("SELECT CURRENT_ROLE()").collect()

[Row(CURRENT_ROLE()='ACCOUNTADMIN')]

In [43]:
# Create a snowpark dataframe 
application_record_sdf = session.table("APPLICATION_RECORD")
credit_record_sdf = session.table("CREDIT_RECORD")
print("Application table size: ", application_record_sdf.count(), 
      "\n Credit table size: ", credit_record_sdf.count())

Application table size:  438557 
 Credit table size:  1048575


In [44]:
credit_record_sdf.show(5)

-----------------------------------------
|"ID"     |"MONTHS_BALANCE"  |"STATUS"  |
-----------------------------------------
|5001711  |0                 |X         |
|5001711  |-1                |0         |
|5001711  |-2                |0         |
|5001711  |-3                |0         |
|5001712  |0                 |C         |
-----------------------------------------



In [45]:
# We will create a new column, TARGET, that will have a 1 for high-risk and 0 for low-risk.
credit_record_sdf = (credit_record_sdf
                     .group_by("ID")
                     .agg(F.sum(F.iff(F.col("STATUS").in_(['2', '3', '4', '5']), 1, 0)).as_("CNT_LATE"))
                     .with_column("TARGET", F.when(F.col("CNT_LATE")>0, 1).otherwise(0))
                     .drop("CNT_LATE"))
credit_record_sdf.show(5)

----------------------
|"ID"     |"TARGET"  |
----------------------
|5001711  |0         |
|5001712  |0         |
|5001713  |0         |
|5001714  |0         |
|5001715  |0         |
----------------------



In [46]:
# Join credit_record data with application_record data
joined_sdf = application_record_sdf.join(credit_record_sdf, "ID", "inner")
joined_sdf.show(5)
joined_sdf.count()
# Drop duplicates
joined_sdf = joined_sdf.drop_duplicates("ID")
joined_sdf.count()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"     |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"  |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"  |"DAYS_EMPLOYED"  |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"  |"FLAG_PHONE"  |"FLAG_EMAIL"  |"OCCUPATION_TYPE"  |"CNT_FAM_MEMBERS"  |"TARGET"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

36457

In [47]:
# Select a few columns for modeling 
cols_numerical = ["AMT_INCOME_TOTAL", "DAYS_EMPLOYED", "FLAG_MOBIL", "CNT_FAM_MEMBERS", "TARGET"]
cols_categorical = ["CODE_GENDER", "NAME_HOUSING_TYPE", "OCCUPATION_TYPE"]
joined_sdf = joined_sdf[cols_numerical+cols_categorical]
joined_sdf.show(5)
joined_sdf.count()

------------------------------------------------------------------------------------------------------------------------------------------------
|"AMT_INCOME_TOTAL"  |"DAYS_EMPLOYED"  |"FLAG_MOBIL"  |"CNT_FAM_MEMBERS"  |"TARGET"  |"CODE_GENDER"  |"NAME_HOUSING_TYPE"  |"OCCUPATION_TYPE"  |
------------------------------------------------------------------------------------------------------------------------------------------------
|90000.0             |365243           |1             |2.0                |0         |F              |House / apartment    |NULL               |
|90000.0             |-735             |1             |4.0                |0         |M              |House / apartment    |Laborers           |
|135000.0            |-397             |1             |3.0                |0         |F              |House / apartment    |Accountants        |
|216000.0            |-1457            |1             |3.0                |0         |M              |Rented apartment     |Labore

36457

In [48]:
# Perform one-hot-encoding for categorical columsn 
my_ohe_encoder = OneHotEncoder(input_cols=cols_categorical, output_cols=cols_categorical, drop_input_cols=True)
prepared_sdf = my_ohe_encoder.fit(joined_sdf).transform(joined_sdf)

prepared_sdf.show(5)
prepared_sdf.columns

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CODE_GEND

['CODE_GENDER_F',
 'CODE_GENDER_M',
 '"NAME_HOUSING_TYPE_Co-op apartment"',
 '"NAME_HOUSING_TYPE_House / apartment"',
 '"NAME_HOUSING_TYPE_Municipal apartment"',
 '"NAME_HOUSING_TYPE_Office apartment"',
 '"NAME_HOUSING_TYPE_Rented apartment"',
 '"NAME_HOUSING_TYPE_With parents"',
 '"OCCUPATION_TYPE_Accountants"',
 '"OCCUPATION_TYPE_Cleaning staff"',
 '"OCCUPATION_TYPE_Cooking staff"',
 '"OCCUPATION_TYPE_Core staff"',
 '"OCCUPATION_TYPE_Drivers"',
 '"OCCUPATION_TYPE_HR staff"',
 '"OCCUPATION_TYPE_High skill tech staff"',
 '"OCCUPATION_TYPE_IT staff"',
 '"OCCUPATION_TYPE_Laborers"',
 '"OCCUPATION_TYPE_Low-skill Laborers"',
 '"OCCUPATION_TYPE_Managers"',
 '"OCCUPATION_TYPE_Medicine staff"',
 '"OCCUPATION_TYPE_Private service staff"',
 '"OCCUPATION_TYPE_Realty agents"',
 '"OCCUPATION_TYPE_Sales staff"',
 '"OCCUPATION_TYPE_Secretaries"',
 '"OCCUPATION_TYPE_Security staff"',
 '"OCCUPATION_TYPE_Waiters/barmen staff"',
 '"OCCUPATION_TYPE_None"',
 'AMT_INCOME_TOTAL',
 'DAYS_EMPLOYED',
 'FLAG_MO

In [49]:
# Cleaning column names to make it easier for future referencing 

import re 

cols = prepared_sdf.columns 
print("old columns: ", cols)
for old_col in cols:
    new_col = re.sub(r'[^a-zA-Z0-9_]', '', old_col)
    new_col = new_col.upper()
    prepared_sdf = prepared_sdf.rename(F.col(old_col), new_col)


prepared_sdf.columns

old columns:  ['CODE_GENDER_F', 'CODE_GENDER_M', '"NAME_HOUSING_TYPE_Co-op apartment"', '"NAME_HOUSING_TYPE_House / apartment"', '"NAME_HOUSING_TYPE_Municipal apartment"', '"NAME_HOUSING_TYPE_Office apartment"', '"NAME_HOUSING_TYPE_Rented apartment"', '"NAME_HOUSING_TYPE_With parents"', '"OCCUPATION_TYPE_Accountants"', '"OCCUPATION_TYPE_Cleaning staff"', '"OCCUPATION_TYPE_Cooking staff"', '"OCCUPATION_TYPE_Core staff"', '"OCCUPATION_TYPE_Drivers"', '"OCCUPATION_TYPE_HR staff"', '"OCCUPATION_TYPE_High skill tech staff"', '"OCCUPATION_TYPE_IT staff"', '"OCCUPATION_TYPE_Laborers"', '"OCCUPATION_TYPE_Low-skill Laborers"', '"OCCUPATION_TYPE_Managers"', '"OCCUPATION_TYPE_Medicine staff"', '"OCCUPATION_TYPE_Private service staff"', '"OCCUPATION_TYPE_Realty agents"', '"OCCUPATION_TYPE_Sales staff"', '"OCCUPATION_TYPE_Secretaries"', '"OCCUPATION_TYPE_Security staff"', '"OCCUPATION_TYPE_Waiters/barmen staff"', '"OCCUPATION_TYPE_None"', 'AMT_INCOME_TOTAL', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'CNT_FAM_

['CODE_GENDER_F',
 'CODE_GENDER_M',
 'NAME_HOUSING_TYPE_COOPAPARTMENT',
 'NAME_HOUSING_TYPE_HOUSEAPARTMENT',
 'NAME_HOUSING_TYPE_MUNICIPALAPARTMENT',
 'NAME_HOUSING_TYPE_OFFICEAPARTMENT',
 'NAME_HOUSING_TYPE_RENTEDAPARTMENT',
 'NAME_HOUSING_TYPE_WITHPARENTS',
 'OCCUPATION_TYPE_ACCOUNTANTS',
 'OCCUPATION_TYPE_CLEANINGSTAFF',
 'OCCUPATION_TYPE_COOKINGSTAFF',
 'OCCUPATION_TYPE_CORESTAFF',
 'OCCUPATION_TYPE_DRIVERS',
 'OCCUPATION_TYPE_HRSTAFF',
 'OCCUPATION_TYPE_HIGHSKILLTECHSTAFF',
 'OCCUPATION_TYPE_ITSTAFF',
 'OCCUPATION_TYPE_LABORERS',
 'OCCUPATION_TYPE_LOWSKILLLABORERS',
 'OCCUPATION_TYPE_MANAGERS',
 'OCCUPATION_TYPE_MEDICINESTAFF',
 'OCCUPATION_TYPE_PRIVATESERVICESTAFF',
 'OCCUPATION_TYPE_REALTYAGENTS',
 'OCCUPATION_TYPE_SALESSTAFF',
 'OCCUPATION_TYPE_SECRETARIES',
 'OCCUPATION_TYPE_SECURITYSTAFF',
 'OCCUPATION_TYPE_WAITERSBARMENSTAFF',
 'OCCUPATION_TYPE_NONE',
 'AMT_INCOME_TOTAL',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'CNT_FAM_MEMBERS',
 'TARGET']

In [50]:
# Split the data and save the train and test sets as tables in Snowflake 
snowdf_train, snowdf_test, snowdf_processed = prepared_sdf.random_split([0.8, 0.1, 0.1], seed=99)
snowdf_train.write.mode("overwrite").save_as_table("CREDIT_DEFAULT_TRAIN")
snowdf_test.write.mode("overwrite").save_as_table("CREDIT_DEFAULT_TEST")
snowdf_processed.write.mode("overwrite").save_as_table("CREDIT_DEFAULT_PROCESSED")

# Reset the warehouse size back to xsmall and close the session 

In [51]:
session.sql("ALTER WAREHOUSE JING_TEST_WH SET WAREHOUSE_SIZE='XSMALL'").collect()

session.close()